In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.preprocessing import image
import os
import matplotlib.pyplot as plt
import cv2

In [2]:
img_dims = 32
batch_size = 32

### Model 1

In [3]:
classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), 
                      input_shape = (img_dims, img_dims, 3), 
                      activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [4]:
input_path = 'Dataset/chest_xray/'
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory(directory=input_path+'train',
target_size = (img_dims, img_dims),
batch_size = 1,
class_mode = 'binary')
test_set = test_datagen.flow_from_directory(directory=input_path+'test',
target_size = (img_dims, img_dims),
batch_size = 1,
class_mode = 'binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [5]:
classifier.compile(optimizer = 'adam', 
                   loss = 'binary_crossentropy',
                   metrics = ['accuracy'])

In [6]:
epochs=10
hist = classifier.fit(
           training_set,
           steps_per_epoch = training_set.samples//batch_size-1,
           epochs=10, validation_data=test_set, 
           validation_steps= test_set.samples)

Epoch 1/10
162/162 [==============================] - 35s 109ms/step - loss: 1.0082 - accuracy: 0.7371 - val_loss: 0.9004 - val_accuracy: 0.6250
Epoch 2/10
162/162 [==============================] - 11s 69ms/step - loss: 0.5072 - accuracy: 0.8076 - val_loss: 0.6482 - val_accuracy: 0.6250
Epoch 3/10
162/162 [==============================] - 10s 65ms/step - loss: 0.5343 - accuracy: 0.7799 - val_loss: 0.7328 - val_accuracy: 0.4551
Epoch 4/10
162/162 [==============================] - 11s 65ms/step - loss: 0.5613 - accuracy: 0.6786 - val_loss: 0.4690 - val_accuracy: 0.7949
Epoch 5/10
162/162 [==============================] - 11s 66ms/step - loss: 0.4205 - accuracy: 0.8190 - val_loss: 0.4247 - val_accuracy: 0.8606
Epoch 6/10
162/162 [==============================] - 11s 66ms/step - loss: 0.4009 - accuracy: 0.8615 - val_loss: 0.4318 - val_accuracy: 0.8093
Epoch 7/10
162/162 [==============================] - 11s 65ms/step - loss: 0.4268 - accuracy: 0.8280 - val_loss: 0.3725 - val_accuracy

In [7]:
test_data = []
test_labels = []
for cond in ['/NORMAL/', '/PNEUMONIA/']:
        for img in (os.listdir(input_path + 'test' + cond)):
            
            img = plt.imread(input_path+'test'+cond+img)
            img = cv2.resize(img, (img_dims, img_dims))
            img = np.dstack([img, img, img])
            img = img.astype('float32') / 255
            if cond=='/NORMAL/':
                label = 0
            elif cond=='/PNEUMONIA/':
                label = 1
            test_data.append(img)
            test_labels.append(label)
        
test_data = np.array(test_data)
test_labels = np.array(test_labels)
from sklearn.metrics import accuracy_score, confusion_matrix

preds = classifier.predict(test_data)

acc = accuracy_score(test_labels, np.round(preds))*100
cm = confusion_matrix(test_labels, np.round(preds))
tn, fp, fn, tp = cm.ravel()

print('CONFUSION MATRIX ------------------')
print(cm)

print('\nTEST METRICS ----------------------')
precision = tp/(tp+fp)*100
recall = tp/(tp+fn)*100
print('Accuracy: {}%'.format(acc))
print('Precision: {}%'.format(precision))
print('Recall: {}%'.format(recall))
print('F1-score: {}'.format(2*precision*recall/(precision+recall)))


CONFUSION MATRIX ------------------
[[156  78]
 [ 25 365]]

TEST METRICS ----------------------
Accuracy: 83.49358974358975%
Precision: 82.39277652370203%
Recall: 93.58974358974359%
F1-score: 87.63505402160864


### Model 2

In [8]:
classifier1 = Sequential()
classifier1.add(Conv2D(32, (3, 3), input_shape = (img_dims, img_dims, 3), activation = 'relu'))
classifier1.add(MaxPooling2D(pool_size = (2, 2)))
classifier1.add(Conv2D(64, (3, 3), input_shape = (img_dims, img_dims, 3), activation = 'relu'))
classifier1.add(MaxPooling2D(pool_size = (2, 2)))
classifier1.add(Flatten())
classifier1.add(Dense(units = 128, activation = 'relu'))
classifier1.add(Dense(units = 1, activation = 'sigmoid'))

In [9]:
classifier1.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [10]:
epochs=10
hist1 = classifier1.fit(
           training_set,steps_per_epoch = training_set.samples//batch_size-1,
           epochs=10, validation_data=test_set, 
           validation_steps= test_set.samples)

Epoch 1/10
162/162 [==============================] - 11s 65ms/step - loss: 0.6545 - accuracy: 0.6697 - val_loss: 0.6636 - val_accuracy: 0.6250
Epoch 2/10
162/162 [==============================] - 11s 67ms/step - loss: 0.6061 - accuracy: 0.7373 - val_loss: 0.6603 - val_accuracy: 0.6250
Epoch 3/10
162/162 [==============================] - 10s 63ms/step - loss: 0.5701 - accuracy: 0.7903 - val_loss: 0.8976 - val_accuracy: 0.6250
Epoch 4/10
162/162 [==============================] - 10s 62ms/step - loss: 0.5079 - accuracy: 0.7810 - val_loss: 0.4944 - val_accuracy: 0.8349
Epoch 5/10
162/162 [==============================] - 10s 62ms/step - loss: 0.5575 - accuracy: 0.7845 - val_loss: 0.5276 - val_accuracy: 0.8253
Epoch 6/10
162/162 [==============================] - 10s 63ms/step - loss: 0.4742 - accuracy: 0.8208 - val_loss: 0.6343 - val_accuracy: 0.6378
Epoch 7/10
162/162 [==============================] - 10s 62ms/step - loss: 0.4525 - accuracy: 0.8178 - val_loss: 0.3955 - val_accuracy:

In [11]:
preds = classifier1.predict(test_data)

acc = accuracy_score(test_labels, np.round(preds))*100
cm = confusion_matrix(test_labels, np.round(preds))
tn, fp, fn, tp = cm.ravel()

print('CONFUSION MATRIX ------------------')
print(cm)

print('\nTEST METRICS ----------------------')
precision = tp/(tp+fp)*100
recall = tp/(tp+fn)*100
print('Accuracy: {}%'.format(acc))
print('Precision: {}%'.format(precision))
print('Recall: {}%'.format(recall))
print('F1-score: {}'.format(2*precision*recall/(precision+recall)))


CONFUSION MATRIX ------------------
[[177  57]
 [ 41 349]]

TEST METRICS ----------------------
Accuracy: 84.2948717948718%
Precision: 85.96059113300493%
Recall: 89.48717948717949%
F1-score: 87.68844221105527
